In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import re
import pandas as pd
import xml.etree.ElementTree as ET
from urllib.request import urlopen


from lxml import etree
import urllib.request
import zipfile
from nltk.tokenize import word_tokenize, sent_tokenize

In [24]:
from gensim.models import Word2Vec, KeyedVectors

In [6]:
!nvidia-smi

Sat Mar 27 09:05:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 1. 훈련데이터 다운로드

In [4]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!ls -l '/content/drive/My Drive/Colab Notebooks/Natural_Language_Processing/data'

total 15660
-rw------- 1 root root 16035163 Mar 27 02:02 ted_en-20160408.zip


In [7]:
url = '/content/drive/My Drive/Colab Notebooks/Natural_Language_Processing/data/ted_en-20160408.zip'

with zipfile.ZipFile(url , 'r') as z:
  target_text = etree.parse(z.open('ted_en-20160408.xml' , 'r'))
  parse_text = '\n'.join(target_text.xpath('//content/text()'))

In [8]:
print(len(parse_text))

24222849


In [9]:
parse_text[:300]

"Here are two reasons companies fail: they only do more of the same, or they only do what's new.\nTo me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation. Both are necessary, but it can be too much of a good thing.\nConsider Facit"

## 2. 데이터 전처리

### 1) 괄호로 구성된 내용 제거

In [12]:
# (content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분 제거)
content_text = re.sub(r'\([^)]*\)' , '' , parse_text)
# 정규표현식의 ^: 제외
# 괄호 사이에 든, 괄호를 제외한 모든 문자
# (hello), () 등 괄호가 든 모든 문자 삭제
# 일정 패턴 안에 포함된 모든 것을 지우고싶을 때 이렇게 사용하자

### 2) 입력 코퍼스에 대해 문장 토큰화

In [10]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [13]:
# NLTK 이용, 문장 단위로 쪼갬
sent_text = sent_tokenize(content_text)

### 3) 구두점 제거, 대문자를 소문자로 변환

In [14]:
normalized_text = []

for string in sent_text :
  tokens = re.sub(r'[^a-z0-9]+', ' ', string.lower())
  normalized_text.append(tokens)

# [^a-z0-9]+ : 알파벳이나 숫자 빼고 한번 이상 반복되는 모든 것

In [15]:
# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행
# 각 문장을 word_tokenize 하면 list 안에 list로 들어감
result = [word_tokenize(sentence) for sentence in normalized_text]

### 4) 전처리 수행 후 샘플의 개수

In [17]:
print('총 샘플의 개수: {}' .format(len(result)))

총 샘플의 개수: 273424


In [18]:
# 샘플 3개 출력
for line in result[:3] :
  print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


## 3. Word2Vec 훈련시키기

In [22]:
%%time

model = Word2Vec(sentences = result,
                 size = 100,      # 밀집벡터의 크기. 워드벡터의 특징값(임베딩 된 벡터의 차원)
                                  # 한 단어가 들어가면 100개의 밀집벡터로 나타남
                                  # 밀집벡터가 클수록 좀 더 정밀하게 벡터로 나타냄
                 window = 5,      # 맥락벡터의 크기. 컨텍스트 윈도우 크기 (앞뒤 5단어)
                 min_count = 5,   # 단어 최소 빈도수 제한(빈도가 적은 단어들은 학습하지 않는다)
                 workers = 4,     # 학습을 위한 프로세스 수 (내부적으로 멀티쓰레드 사용)
                                  # GPU에 의미가 있음
                 sg = 0)          # skip-gram: 0(CBOW), 1(Skip-gram)

# model로 학습까지 됨

CPU times: user 49.3 s, sys: 363 ms, total: 49.7 s
Wall time: 28.1 s


- model.wv.most_similar: 가장 유사한 단어들을 출력

In [23]:
# man과 유사한 단어 출력
model_result = model.wv.most_similar('man')
print(model_result)
# 내부적으로 코사인 유사도를 구함

[('woman', 0.8382266759872437), ('guy', 0.8277359008789062), ('lady', 0.7799607515335083), ('gentleman', 0.7605015635490417), ('boy', 0.7373381853103638), ('girl', 0.7265478372573853), ('poet', 0.7214939594268799), ('soldier', 0.7167720794677734), ('kid', 0.686995267868042), ('david', 0.6737528443336487)]


## 4. Word2Vec 모델 저장하고 로드하기

In [31]:
model.wv.save_word2vec_format('eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format('eng_w2v') # 모델 로드

In [32]:
model_result = loaded_model.most_similar('man')
print(model_result)

[('woman', 0.8382266759872437), ('guy', 0.8277359008789062), ('lady', 0.7799607515335083), ('gentleman', 0.7605015635490417), ('boy', 0.7373381853103638), ('girl', 0.7265478372573853), ('poet', 0.7214939594268799), ('soldier', 0.7167720794677734), ('kid', 0.686995267868042), ('david', 0.6737528443336487)]
